In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Tue Jun 29 00:02:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

     |████████████████████████████████| 2.5MB 4.2MB/s 
     |████████████████████████████████| 3.3MB 23.1MB/s 
     |████████████████████████████████| 901kB 47.1MB/s 


In [4]:
import pandas as pd

In [6]:
# df=pd.read_csv('/content/training_data_distil_bert_clause_data+300_incorrect_predicted_data_wet_ic.csv')
df=pd.read_csv('/content/Cleaned_balanced_data.csv')

In [7]:
df.head()

,Unnamed: 0,clause_text,clause_type,clause_text_clean,labels
0,0,In the event that any obligation contained he...,Force Majeure,event obligation contained herein not fulfille...,1
1,1,Nuix will not be liable for any failure or de...,Force Majeure,nuix not liable failure delay performing oblig...,1
2,2,If by reason of force majeure either party is...,Force Majeure,reason force majeure either party unable whole...,1
3,3,In no event shall either party be liable or d...,Force Majeure,no event shall either party liable deemed defa...,1
4,4,Notwithstanding anything to the contrary cont...,Force Majeure,notwithstanding anything contrary contained he...,1


In [8]:
df['clause_type'].unique()

array(['Force Majeure', 'Liquidated Damages',
       'RoW Permissions and Clearances', 'Payment related clauses',
       'Termination/Suspension', 'Variations', 'Dispute Resolution'],
      dtype=object)

In [ ]:
# df=df.rename(columns={'Label Name':'Label'})

In [ ]:
# df=df.drop(columns={'Annotated By','Doc Title','Page Number','Annotated By','Unnamed: 0','prediction','Confidence_score','new_model_prediction'})

In [9]:
df=df.drop(columns={'labels','Unnamed: 0'})

In [ ]:
# df=pd.read_csv('/content/clause_data.csv')
# df['Label'].unique()

In [ ]:
df.head()

,clause_text,clause_type,clause_text_clean
0,In the event that any obligation contained he...,Force Majeure,event obligation contained herein not fulfille...
1,Nuix will not be liable for any failure or de...,Force Majeure,nuix not liable failure delay performing oblig...
2,If by reason of force majeure either party is...,Force Majeure,reason force majeure either party unable whole...
3,In no event shall either party be liable or d...,Force Majeure,no event shall either party liable deemed defa...
4,Notwithstanding anything to the contrary cont...,Force Majeure,notwithstanding anything contrary contained he...


In [10]:

import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset

In [ ]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df
# data_clean = clean_text(train_data, 'text', 'text_clean')
# data_clean.head()

In [ ]:
# df=pd.read_csv('/content/clause_data.csv')

In [ ]:


# df=clean_text(df,'Segment Text','text')

In [ ]:
# df=df.drop(columns='Segment Text')

In [11]:
df.head()

,clause_text,clause_type,clause_text_clean
0,In the event that any obligation contained he...,Force Majeure,event obligation contained herein not fulfille...
1,Nuix will not be liable for any failure or de...,Force Majeure,nuix not liable failure delay performing oblig...
2,If by reason of force majeure either party is...,Force Majeure,reason force majeure either party unable whole...
3,In no event shall either party be liable or d...,Force Majeure,no event shall either party liable deemed defa...
4,Notwithstanding anything to the contrary cont...,Force Majeure,notwithstanding anything contrary contained he...


In [12]:
df=df.rename(columns={'clause_text_clean':'text'})

In [13]:
df['text']=df['text'].astype('str')

In [ ]:
# df=df.rename(columns={'Segment Text':'text'})

In [14]:
df=df.rename(columns={'clause_type':'Label'})

In [15]:
df['Label'].value_counts()

Termination/Suspension            485
Dispute Resolution                484
Variations                        481
Liquidated Damages                478
Force Majeure                     472
Payment related clauses           471
RoW Permissions and Clearances    345
Name: Label, dtype: int64

In [17]:

possible_labels = df.Label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Dispute Resolution': 6,
 'Force Majeure': 0,
 'Liquidated Damages': 1,
 'Payment related clauses': 3,
 'RoW Permissions and Clearances': 2,
 'Termination/Suspension': 4,
 'Variations': 5}

In [18]:
df['label'] = df.Label.replace(label_dict)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Label', 'label', 'data_type']).count()

clause_text  text
Label                          label data_type                   
Dispute Resolution             6     train              412   412
                                     val                 72    72
Force Majeure                  0     train              401   401
                                     val                 71    71
Liquidated Damages             1     train              406   406
                                     val                 72    72
Payment related clauses        3     train              400   400
                                     val                 71    71
RoW Permissions and Clearances 2     train              293   293
                                     val                 52    52
Termination/Suspension         4     train              412   412
                                     val                 73    73
Variations                     5     train              409   409
                                     val                 72    72

In [20]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')



In [21]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=400, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [22]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=400, 
    return_tensors='pt'
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [23]:
encoded_data_train.keys()

dict_keys(['input_ids', 'attention_mask'])

In [24]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)
 

In [25]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [26]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [28]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-7)
                  
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [29]:
from sklearn.metrics import f1_score


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [30]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device("cuda:0")
model = model.to(device)
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/distil_bert_clause_data_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.9736872804269456
Validation loss: 0.2732979104884209
F1 Score (Weighted): 0.9626698924290665



Epoch 2
Training loss: 0.24107168566936638
Validation loss: 0.15864769765926945
F1 Score (Weighted): 0.9688552265573862



In [31]:

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/distil_bert_clause_data_2.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

Class: Force Majeure
Accuracy: 71/71

Class: Liquidated Damages
Accuracy: 68/72

Class: RoW Permissions and Clearances
Accuracy: 52/52

Class: Payment related clauses
Accuracy: 69/71

Class: Termination/Suspension
Accuracy: 67/73

Class: Variations
Accuracy: 70/72

Class: Dispute Resolution
Accuracy: 71/72

